# Opérations sur les tenseurs avec einops

L'idée est de :
1. manipuler les tenseurs et les multiplier
2. visualiser l'extraction de
  - patchs dans des images
  - tubelets dans des videos
  
Pour bien comprendre comment on les réarrange spatialement, temporellement

les manipulations de dimensions peuvent se faire :
- avec torch (squeeze, stack, permute, reshape, transpose)
- avec **einops** : une librairie qui rend les choses plus lisibles et plus stables

Ce tuto utilise le plus souvent **einops**.

Déja, on fait les install et les imports

In [ ]:
!pip install torchviz

In [ ]:
import numpy as np
import torch
from torchviz import make_dot

import einops

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Matrice x tenseur d'ordre 3

Ici, on commence ce qui m'intéresse, car c'est ce qui se passe quand on passe
une phrase dans un transformer.

- chaque mot (token) est encodé en un vecteur de shape $(s_{token},)$. Ci dessous : $s_{token} = 2$
- Une phrase est une sequence de token. c'est une matrice de shape $(s_{seq},s_{token})$. Ci dessous : ($s_{seq} =3$)
- un batch de phrase est un tenseur de shape $(s_{batch},s_{seq},s_{token})$. Ci dessous $s_{batch} = 4$

Prenons un MLP qui travaillerait sur chaque token indépendamment,
pour chaque composante du token, il calcule une nouvelle sortie.

Si la dimension de sortie est aussi $s_{token}$, sa matrice est de shape $(s_{token},s_{token})$

- Si on applique cette matrice à un token (vecteur), la sortie est un vecteur, tout se passe comme prévu.
- Si on applique cette matrice à une sequence de token (une matrice), la sortie est le resultat d'une multiplication matricielle. Il faut **faire attention à ne pas faire d'opérations entre composantes de tokens différents**.

On travaille bien sous forme $input \times Matrice$ , et tout va bien.

Pour être bien propre, on va mettre les opérations effectuées sur chaque token en **ligne** dans la matrice initiale, puis **la transposer**.

In [ ]:

# prenons une matrice qui calcule, pour un token, la somme et la différence de ses composantes
W = torch.Tensor([[1,1],[1,-1]]).float().to(device)
# on transpose car on multiplie par la gauche.
W = einops.rearrange(W, "h w -> w h" )
print ("W",W)

# le premier token de la premiere phrase est [1,2]. le second est [3,4]...
# le premier token de la seconde phrase est [11,12]. le second est [13,14]
phrase = torch.tensor([[1,2],[3,4],[5,6]]).float().to(device)
print("\nshape d'une phrase",phrase.shape)
print("\nexemple de phrase\n",phrase)

print ("\n==========BATCHS de phrase (PAR LA GAUCHE) ===============\n")
# on stack sur b.
batch = einops.rearrange([phrase,phrase + 10,phrase + 100,phrase + 1000], "b nseq embed -> b nseq embed")
print("\nun batch de 4 phrases\n",batch.int())

res = batch @ W
print("resultat\n",res.int())


W tensor([[ 1.,  1.],
        [ 1., -1.]])

shape d'une phrase torch.Size([3, 2])

exemple de phrase
 tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])

==========BATCHS de phrase (PAR LA GAUCHE) ===============


un batch de 4 phrases
 tensor([[[   1,    2],
         [   3,    4],
         [   5,    6]],

        [[  11,   12],
         [  13,   14],
         [  15,   16]],

        [[ 101,  102],
         [ 103,  104],
         [ 105,  106]],

        [[1001, 1002],
         [1003, 1004],
         [1005, 1006]]], dtype=torch.int32)
resultat
 tensor([[[   3,   -1],
         [   7,   -1],
         [  11,   -1]],

        [[  23,   -1],
         [  27,   -1],
         [  31,   -1]],

        [[ 203,   -1],
         [ 207,   -1],
         [ 211,   -1]],

        [[2003,   -1],
         [2007,   -1],
         [2011,   -1]]], dtype=torch.int32)


## INTERPRETATIONS DE CES MULTIPLICATIONS : BROADCASTING

lu quelque part :
> "The matrix multiplication(s) are done between the last two dimensions. The remaining first three dimensions are broadcast and are ‘batch’"

testons ca. Ici, on a

- un tenseur $W_{batch}$ de shape $[2,2,2]$ qui représente 2 matrices empilées
- un tenseur $batch$ de shape $[4,2,3,2]$ qui représente nos inputs. on va les imaginer comme 4 images de 2 canaux, 3 lignes 2 colonnes.

Pour bien s'assurer de ce que l'on fait, vu les égalités de longueur des shape, précisons l'ordre des canaux pour $batch$ : $[B, C, H, W]$

Pour la matrice, la shape est $[C, H, W]$

On calcule $ batch \times W_{batch}$.

Le résultat est surprenant :

- la premiere matrice de $W_{batch}$ multiplie le premier canal d'une image.
- la seconde matrice de $W_{batch}$ multiplie le second canal d'une image.
- ces opérations sont répétées pour chaque image



Ceci est lié au fait que la multiplication **broadcaste** les données : https://docs.pytorch.org/docs/stable/notes/broadcasting.html#broadcasting-semantics

le **broadcast** dans la multiplication matricielle a des comportements differents en fonction des tailles des dimensions respectives des tenseurs.

Pour savoir s'il faut et si on peut broadcaster, **on regarde les dimensions en partant de la fin et en remontant vers le début**

1. Pour *matmult* : les deux dernieres dimensions doivent être compatibles pour une multiplication matricielle.

2. comme la troisieme dimension en partant de la fin est de même taille dans les deux tenseurs, tout se passe comme si on appliquait les calculs dans les dimensions suivantes de façon parallele. Ceci explique comment on ferait un calcul d'un tenseur de shape $[C,H,W] \times$ un tenseur de shape $[C,H,W]$ :
**chaque matrice $[H,W]$ des inputs est multipliée par la matrice $[H,W]$ correspondante du tenseur.**

3. Pour la partie batch, la 4eme dimension en partant de la fin n'existe pas dans W. Pas de problème, le broadcast va la créer et dupliquer les données.
Pour imaginer ca, disons qu'un scalaire $a$, ca peut se broadcaster en un vecteur $[a,a,a,a]$. **Notons que** $a$ **pourrait être un vecteur ou une matrice, ca marcherait pareil**. Donc le tenseur $W_{batch}$ va être étendu pour
appliquer l'opération 2 pour chaque "phrase" de $batch$

4. on s'en servira plus loin, mais une dimension de 1 peut aussi se broadcaster par duplication. Pour imaginer ca, disons qu'une matrice 1,2 telle que $[[1,2,3]]$ peut se broadcaster en une matrice 3,2 : $[[1,2,3],[1,2,3],[1,2,3]]$


In [ ]:
# on prépare une liste de matrice.
# W1 va conserver la premiere composante d'un token.
W1 = torch.tensor([[1,0],[0,0]]).float().to(device)
W1 = einops.rearrange(W1, "h w -> w h" )

# Pour X, on va creer un tenseur [b C H W] (4 2 3 2) a partir des données précédentes
# le second canal d'une image est simplement l'opposé du premier
batch_im = einops.rearrange([batch,-batch], "c b h w -> b c h w")

# on batche les matrices
W_batch = einops.rearrange([W,W1],"b h w -> b h w")
print("\nW_batch\n",W_batch)

print("batch.shape",batch_im.shape,"W.shape",W_batch.shape)

res = batch_im @ W_batch

print("\nbatch d'images\n",batch_im.int())

print("\nres\n",res.int())

print("batch.shape",batch_im.shape,"W.shape",W_batch.shape,"res.shape :",res.shape)


W_batch
 tensor([[[ 1.,  1.],
         [ 1., -1.]],

        [[ 1.,  0.],
         [ 0.,  0.]]])
batch.shape torch.Size([4, 2, 3, 2]) W.shape torch.Size([2, 2, 2])

batch d'images
 tensor([[[[    1,     2],
          [    3,     4],
          [    5,     6]],

         [[   -1,    -2],
          [   -3,    -4],
          [   -5,    -6]]],


        [[[   11,    12],
          [   13,    14],
          [   15,    16]],

         [[  -11,   -12],
          [  -13,   -14],
          [  -15,   -16]]],


        [[[  101,   102],
          [  103,   104],
          [  105,   106]],

         [[ -101,  -102],
          [ -103,  -104],
          [ -105,  -106]]],


        [[[ 1001,  1002],
          [ 1003,  1004],
          [ 1005,  1006]],

         [[-1001, -1002],
          [-1003, -1004],
          [-1005, -1006]]]], dtype=torch.int32)

res
 tensor([[[[    3,    -1],
          [    7,    -1],
          [   11,    -1]],

         [[   -1,     0],
          [   -3,     0],
          [   

### Broadcasting reloaded

Vu la doc, on va tester ceci : appliquer une deuxieme transformation à chacun de nos tokens.

On va modifier les inputs pour creer une dimension juste avant les deux dernieres, pour avoir : $[B, C,1, H, W]$

la matrice est de shape : [2,H,W]

On calcule X @ W

cette fois ci, chaque matrice H,W des données passe dans chacune des deux matrices de traitement.

Le broadcasting a en fait dupliqué les données des dimensions suivantes dans la dimension de taille 1 pour égaler la taille 2. Puis on applique la strat précédente.

**Ce sont des manipulations comme celle ci (et celles d'avant) qui permettent de faire de l'attention spatiale ou temporelle à moindre frais** : on va selectionner des vecteurs pertinents par permutation, pour les batcher.

**a noter : dans un cadre opérationnel, vu les résultats ci dessous, il faudrait peut être permuter les dimensions du résultat** pour que les 2 calculs effectués sur chaque token soient la deuxieme dimension en partant de la fin.

En l'état, à la sortie, j'ai une shape $[B,C,n_{op},H, W]$


In [ ]:
X_reshaped = einops.rearrange(batch_im, "b c h w -> b c 1 h w")
print ("\nbatch reshaped\n",X_reshaped.int())

res = X_reshaped @ W_batch
print("\nres\n",res.int())

print("X.shape",X_reshaped.shape,"W.shape",W_batch.shape,"res.shape :",res.shape)


batch reshaped
 tensor([[[[[    1,     2],
           [    3,     4],
           [    5,     6]]],


         [[[   -1,    -2],
           [   -3,    -4],
           [   -5,    -6]]]],



        [[[[   11,    12],
           [   13,    14],
           [   15,    16]]],


         [[[  -11,   -12],
           [  -13,   -14],
           [  -15,   -16]]]],



        [[[[  101,   102],
           [  103,   104],
           [  105,   106]]],


         [[[ -101,  -102],
           [ -103,  -104],
           [ -105,  -106]]]],



        [[[[ 1001,  1002],
           [ 1003,  1004],
           [ 1005,  1006]]],


         [[[-1001, -1002],
           [-1003, -1004],
           [-1005, -1006]]]]], dtype=torch.int32)

res
 tensor([[[[[    3,    -1],
           [    7,    -1],
           [   11,    -1]],

          [[    1,     0],
           [    3,     0],
           [    5,     0]]],


         [[[   -3,     1],
           [   -7,     1],
           [  -11,     1]],

          [[   -1,   


## Rearrange dans les images

on a une image, composée de 3 canaux, de taille 2 x 4

On veut faire un paquet de vecteurs. Chaque vecteur correspond à un pixel et contient les 3 canaux.



In [ ]:
# un canal d'une image
imR = torch.tensor([[1,2,3,4],[5,6,7,8]]).float().to(device)
# l'image : le seconde canal est le premier + 10, le troisieme premier -10

im = einops.rearrange([imR,imR+10,imR-10],"c h w -> c h w")

print("\nune image \n",im)
print("\nshape d'une image",im.shape)

# On reshape chaque canal en un vecteur de taille hxw et on transpose...
im_reshaped = einops.rearrange(im,"c h w -> (h w) c")
print("\n après reshape\n")
print(im_reshaped)



une image 
 tensor([[[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.]],

        [[11., 12., 13., 14.],
         [15., 16., 17., 18.]],

        [[-9., -8., -7., -6.],
         [-5., -4., -3., -2.]]])

shape d'une image torch.Size([3, 2, 4])

 après reshape

tensor([[ 1., 11., -9.],
        [ 2., 12., -8.],
        [ 3., 13., -7.],
        [ 4., 14., -6.],
        [ 5., 15., -5.],
        [ 6., 16., -4.],
        [ 7., 17., -3.],
        [ 8., 18., -2.]])


### Idem mais avec des patchs

- on part d'une image à 2 canaux, de taille 6x4 (c h w)
- on va faire des patchs de taille 2x1 (hp x hw) dans cette image.

ca va faire une matrice de patchs de taille $[nph, npw]$ pour un total de $nph \times npw$ patchs


In [ ]:


imR = torch.tensor([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16],[17,18,19,20],[21,22,23,24]]).float().to(device)
im = einops.rearrange([imR,imR+100],"c h w -> c h w")
print(im)


# On reshape en patch de taille 1x2...

sp_i = 3 # taille des patchs en nb lignes
sp_j = 2 #taille des patchs en nb colonnes

# en deux étapes pour bien vérifier
# on crée les patchs en découpant les H et les W
# ca va donner une matrice de patchs
# equivalent à (C,H//hp,hp, W//wp,wp)
im_reshaped = einops.rearrange(im, "c (nph hp) (npw wp) -> nph npw c hp wp", hp=3, wp=2)
print("\nmatrice de patchs constitués comme des images (cx3x2)\n", im_reshaped)

# On fusionne les dimensions finales pour avoir chaque patch sous forme vectorielle
# on fusionne aussi les dimensions initiale pour avoir une liste de patchs
im_reshaped = einops.rearrange(im_reshaped, "nph npw c hp wp -> (nph npw) (c hp wp)")
print("\nliste de patchs constitués comme des vecteurs\n")
print(im_reshaped)

print("\nci dessus, j'ai bien une liste de 4 patchs de vecteurs\n")



tensor([[[  1.,   2.,   3.,   4.],
         [  5.,   6.,   7.,   8.],
         [  9.,  10.,  11.,  12.],
         [ 13.,  14.,  15.,  16.],
         [ 17.,  18.,  19.,  20.],
         [ 21.,  22.,  23.,  24.]],

        [[101., 102., 103., 104.],
         [105., 106., 107., 108.],
         [109., 110., 111., 112.],
         [113., 114., 115., 116.],
         [117., 118., 119., 120.],
         [121., 122., 123., 124.]]])

matrice de patchs constitués comme des images (cx3x2)
 tensor([[[[[  1.,   2.],
           [  5.,   6.],
           [  9.,  10.]],

          [[101., 102.],
           [105., 106.],
           [109., 110.]]],


         [[[  3.,   4.],
           [  7.,   8.],
           [ 11.,  12.]],

          [[103., 104.],
           [107., 108.],
           [111., 112.]]]],



        [[[[ 13.,  14.],
           [ 17.,  18.],
           [ 21.,  22.]],

          [[113., 114.],
           [117., 118.],
           [121., 122.]]],


         [[[ 15.,  16.],
           [ 19.,  20.],



### Manipulations spatio temporelles sur des videos

A partir d'ici, on a compris que si on rajoute une dimension Batch, les opérations se feront sans difficultés sur chacun des items du batch par brodcasting.

On ne travaillera donc plus, pour ces démos, avec des données batchées.

#### Préparation d'une vidéo exemple
On va se faire une video de 8 images.
chaque image a :
- 2 canaux ([1,2...] et [-1, -2...])
- une taille de 6x4
- les différentes images sont calculees comme suit : $im_i = 10^i \times im_1$

Nos données ont une shape : $(frames,C,H,W)$

#### Encodage de cette vidéo

Le code qui suit permettra de changer les tailles des tubelets.

Je note quand meme ci dessous le cas pour lequel les données ont été préparées

- On encode en tubelets (patchs) de taille $[fp,hp,wp] = [2,3,2]$

Chaque tubelet est donc un patch de 2 frames, s'etendant spatialement sur 3x2 pixels.

- Ca fait $(npt\times nph \times npw)$ patchs (4x2x2) patchs (16 patchs au total)
- chaque patch a une shape $[fp, c, hp, hw]$, soit (2,2,3,2)

pour finaliser le traitement,
- je vectorise chaque patch : un vecteur de taille $fp \times c \times hp \times hp$, soit (2x2x3x2) = 24
- je vectorise l'ensemble des patchs : une liste de taille $npt \times nh \times nw$, soit 4x2x2 = 16 patchs

Le but du code suivant est de **jouer avec les tailles de tubelets pour comprendre les manips d'extraction de patchs**



In [ ]:
# une fonction pour faire les patchs
# input est une video [frames, C, H, W]
def extract_patches(inputs,fp=2, hp=3,wp=2):

  # en deux étapes pour bien vérifier
  # on crée les patchs en découpant les frames, les H et les W
  # ca va donner une matrice de patchs
  # equivalent à (f//fp, fp, C,H//hp,hp, W//wp,wp), réorganisés
  inputs_reshaped = einops.rearrange(inputs, "(npt fp) c (nph hp) (npw wp) -> npt nph npw fp c hp wp", fp=fp, hp=hp, wp=wp)

  # On fusionne les dimensions finales pour avoir chaque patch sous forme vectorielle
  # on fusionne aussi les dimensions initiale pour avoir une liste de patchs
  inputs_reshaped = einops.rearrange(inputs_reshaped, "npt nph npw fp c hp wp -> npt nph npw (fp c hp wp)")

  return inputs_reshaped


In [ ]:
h = 6
w = 4
im_1 = torch.tensor([i+1 for i in range(h*w)])
im_1 = im_1.reshape(h,w)

print("im 1\n",im_1)

# canal 2 : - canal 1
im = einops.rearrange([im_1, -im_1],"c h w -> c h w")

# les differentes frames, frame1, 10 * frame1, 100*frame1,...
list_im = [im*(10**(i)) for i in range(8)]

vid = torch.stack(list_im)

print ("\nvid\n", vid)
print ("\nvid shape\n", vid.shape)

hp = 3  # H = 6   =>  on peut prendre 1, 2, 3 ou 6
wp = 2  # W = 4   =>  on peut prendre 1, 2, ou 4
fp = 2  # F = 8   =>  on peut prendre 1, 2, 4, ou 8
patches = extract_patches(vid, hp = hp, wp=wp, fp=fp)
print("\npatches.shape\n",patches.shape)
print("\n patches\n",patches)

print("on a bien dans un patch : frame initiale channel 1, frame initiale channel 2, frame suivante channel 1 frame suivante channel 2")
print("on a bien tous les patchs dans l'ordre : toutes les colonnes, puis toutes les lignes, puis tous les temps")

print("\npatches.shape\n",patches.shape)

im 1
 tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]])

vid
 tensor([[[[         1,          2,          3,          4],
          [         5,          6,          7,          8],
          [         9,         10,         11,         12],
          [        13,         14,         15,         16],
          [        17,         18,         19,         20],
          [        21,         22,         23,         24]],

         [[        -1,         -2,         -3,         -4],
          [        -5,         -6,         -7,         -8],
          [        -9,        -10,        -11,        -12],
          [       -13,        -14,        -15,        -16],
          [       -17,        -18,        -19,        -20],
          [       -21,        -22,        -23,        -24]]],


        [[[        10,         20,         30,         40],
          [        50,         60,         70

parfait. Le tuto suivant consistera a manipuler des inputs pour faire des manips spatio temporelles dedans